In [2]:
from helper import print_sents, accuracy, tokenized_word_sets

**tf.tsv**

always predict A-coref as True and B-coref as False

In [3]:
f_test = open("gap-test.tsv", "r")
result = open("tf.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    result.write("\t".join([attrs[0]] + ['True', 'False']) + "\n")

f_test.close()
result.close()

In [4]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=tf.tsv

Overall recall: 51.8 precision: 45.9 f1: 48.7
		tp 918	fp 1082
		fn 855	tn 1145
Masculine recall: 51.0 precision: 45.3 f1: 48.0
		tp 453	fp 547
		fn 436	tn 564
Feminine recall: 52.6 precision: 46.5 f1: 49.4
		tp 465	fp 535
		fn 419	tn 581
Bias (F/M): 1.03

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:


**closer.tsv**

Predict closer coreference to pronoun as True

In [5]:
f_test = open("gap-test.tsv", "r")
result = open("closer.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    tf = ['True', 'False']
    if abs(int(attrs[3]) - int(attrs[5])) > abs(int(attrs[3]) - int(attrs[8])):
        tf = ['False', 'True']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [6]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=closer.tsv

Overall recall: 50.1 precision: 44.4 f1: 47.1
		tp 888	fp 1112
		fn 885	tn 1115
Masculine recall: 51.6 precision: 45.9 f1: 48.6
		tp 459	fp 541
		fn 430	tn 570
Feminine recall: 48.5 precision: 42.9 f1: 45.5
		tp 429	fp 571
		fn 455	tn 545
Bias (F/M): 0.94

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:


In [7]:
print_sents(random=True)

Pratt claims that her parents turned a blind eye on her addiction issues. In 2017, Pratt launched her own collection for Goddiva. Pratt is the younger sister of Spencer Pratt, and the sister-in-law of Heidi Montag (both of whom are on The Hills). She has struggled with bulimia in the past due to the pressure to be thin stemming from appearing alongside very thin co-stars on the show.

He missed weeks 6 through 8 with a sprained ankle. In week 11 against St. Louis, Williams filled in for Foster, who had injured his elbow, and totaled 20 carries for 114 yards. In a Monday Night Football game against the Philadelphia Eagles, Williams started for an injured DeShaun Foster and managed 74 rushing yards on 17 carries and an impressive 101 receiving yards on 7 catches with a touchdown. He had 501 rushing yards and 1 touchdown on the year.

Sarah Wigglesworth MBE RDI is a British award-winning architect and Professor of Architecture at the University of Sheffield. Wigglesworth founded Sarah Wig

In [8]:
import pandas as pd

In [9]:
train_data = pd.read_csv('gap-development.tsv', sep='\t')
test_data = pd.read_csv('gap-test.tsv', sep='\t')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              2000 non-null   object
 1   Text            2000 non-null   object
 2   Pronoun         2000 non-null   object
 3   Pronoun-offset  2000 non-null   int64 
 4   A               2000 non-null   object
 5   A-offset        2000 non-null   int64 
 6   A-coref         2000 non-null   bool  
 7   B               2000 non-null   object
 8   B-offset        2000 non-null   int64 
 9   B-coref         2000 non-null   bool  
 10  URL             2000 non-null   object
dtypes: bool(2), int64(3), object(6)
memory usage: 144.7+ KB


**subject.tsv**

Predict farther coreference pronoun as True when pronoun is subject pronoun (he, she).
Predict closer coreference pronoun as True when pronoun is not subject pronoun, which is object or possessive pronoun(her, his, him).

In [10]:
pronouns = set(train_data['Pronoun'])
pronouns

{'He', 'Her', 'His', 'She', 'he', 'her', 'him', 'his', 'she'}

In [11]:
result = open("subject.tsv", 'w')

for attrs in test_data.iloc:
    pro, pro_id = attrs[2:4]
    A, A_id = attrs[4:6]
    B, B_id = attrs[7:9]

    if pro.lower() in ['he', 'she']:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['TRUE', 'FALSE']
        else:
            tf = ['FALSE', 'TRUE']
    else:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['FALSE', 'TRUE']
        else:
            tf = ['TRUE', 'FALSE']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [12]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=subject.tsv

Overall recall: 58.3 precision: 51.6 f1: 54.8
		tp 1033	fp 967
		fn 740	tn 1260
Masculine recall: 59.4 precision: 52.8 f1: 55.9
		tp 528	fp 472
		fn 361	tn 639
Feminine recall: 57.1 precision: 50.5 f1: 53.6
		tp 505	fp 495
		fn 379	tn 621
Bias (F/M): 0.96

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:


In [13]:
false_case = 0
for attrs in test_data.iloc:
    if attrs[6] == attrs[9]:
        false_case += 1

false_case/len(test_data)

0.1135

# relation.tsv

draw a relation between two proper nouns with sentence structure

(e.g. Which one is the subject?)

In [14]:
import nltk
sent = "In the next pair, Mikhaylov equalled Grishin's time and they shared Olympic gold."
tagged_sent = nltk.pos_tag(nltk.word_tokenize(sent))

Classify train_data into several categories

1. When words are inside parenthesis
2. When words appear after the pronoun
 * When the pronoun is subject pronoun
 * When the pronoun is object/possessive pronoun
3. When the both word coreference is false
4. When the pronoun is subject pronoun

 * When only one word is subject
 * When both words are subject
 * When both words are not subject

5. When the pronoun is object/possessive pronoun
 * When the pronoun and words are in different sentence
 * When the pronoun and words are in same sentence
 * When only one word is in same sentence with the pronoun
    

In [15]:
import pandas as pd 

train_data = pd.read_csv('gap-development.tsv', sep='\t')

## 1. When words are inside parenthesis
Assume when word is inside parenthesis, coreference is always false

In [16]:
parenthesis_data = []

def isparenthesis(sent, word, offset):
    if sent[offset-1] == "(" and sent[offset+len(word)] == ")":
        return True
    else:
        return False

for row in train_data.iloc:
    if isparenthesis(row['Text'], row['A'], row['A-offset']) or \
       isparenthesis(row['Text'], row['B'], row['B-offset']):
            parenthesis_data.append(row)

In [17]:
parenthesis_data = pd.DataFrame(parenthesis_data, columns=train_data.columns)

In [18]:
print_sents(datas={'manual':True, 'data':parenthesis_data}, num_lines = 10)

Alf and Else's daughter Rita (Una Stubbs) now lives with her husband Mike in his hometown of Liverpool and often visits her parents, although Mike does not appear (as Antony Booth had no interest in reprising the role). Usually, Alf is seen drinking with his friend Arthur (Arthur English) in the local pub. Although his beloved Conservative Party has returned to power, Alf is not happy with Margaret Thatcher being Prime Minister because, according to him, ``a woman's place is in the home''.

Roscoe and his wife had seven sons and three daughters, including William Stanley Roscoe (1782--1843), a poet, Thomas Roscoe (1791--1871), translator from Italian, and Henry (1800--1836), a legal writer who wrote his father's biography. Henry's wife, Maria Roscoe (n*e Fletcher) (1798-1885), was the author of Vittoria Colonna: Her Life and Times, and their son Henry Enfield Roscoe (1833--1915) was a chemist and vice-chancellor of the University of London.

Crime boss Rusty Pirone (Armand Assante) is 

result : 22 out of 31 sentences are predicted correct --> accuracy : 71%

## 2. When words appear after the pronoun
Assume coreference of words coming after the pronoun is false

In [19]:
after_pronoun = []

for row in train_data.iloc:
    if row['Pronoun-offset'] < row['A-offset'] or row['Pronoun-offset'] < row['B-offset']:
        after_pronoun.append(row)

after_pronoun = pd.DataFrame(after_pronoun, columns=train_data.columns)

In [20]:
after_subject_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['he', 'she', 'He', 'She'])]
after_other_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['his', 'her', 'him', 'His', 'Her', 'Him'])]

In [21]:
print_sents(datas={'manual':True, 'data':after_subject_pronoun}, num_lines = 10)

Kaori tells him that the reason she does not want to get close with Shiori is that she does not want to feel the intense pain that will result from Shiori's death. Kaori breaks down after saying this, crying on Yuichi and lamenting the pointlessness of Shiori's birth. After this point, Kaori starts to avoid Shiori even more than usual. When confronted by Yuichi about how she has been acting, Kaori tells him that she has no younger sister.

Isabel Walker is a twenty-something American film-school dropout who decides to visit her step-sister Roxanne, a poet who now lives in France. Isabel believes that she is there to help Roxanne during her pregnancy with her toddler infant, but later realizes that her father and step-mother sent her there so that Roxanne would help the shiftless Isabel gain some direction in life. Shortly after she arrives, Roxanne confides in Isabel that her French husband, Claude-Henri has left her.

Farrar also had a bad day, finishing with the second group on the r

In [22]:
def after_pronoun_predict(data):
    A_pred = []
    B_pred = []
    for row in data.iloc:
        A_value = False if row['A-offset'] > row['Pronoun-offset'] else True
        B_value = False if row['B-offset'] > row['Pronoun-offset'] else True

        A_pred.append(A_value)
        B_pred.append(B_value)

    data['A-pred'] = A_pred
    data['B-pred'] = B_pred

In [23]:
after_pronoun_predict(after_subject_pronoun)
after_pronoun_predict(after_other_pronoun)

<ipython-input-22-ceb8dec2fe17>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['A-pred'] = A_pred
<ipython-input-22-ceb8dec2fe17>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['B-pred'] = B_pred


In [24]:
print(accuracy(after_subject_pronoun))
print(accuracy(after_other_pronoun))

0.411214953271028
0.2133676092544987


Result : It doesn't really matter whether a word comes after the pronoun or not

## 3. When the both word coreference is false

Let's find out what they have in common.

In [25]:
false_cases = 0
for row in train_data.iloc:
    if row['A-coref'] == row['B-coref']:
        false_cases += 1

print(false_cases/len(train_data))
print(false_cases)

0.1005
201


There are more than 10 percent of these kind of cases

In [26]:
falses_data = train_data[(train_data['A-coref'] == False) & (train_data['B-coref'] == False)]

In [27]:
print_sents(datas={'manual':True, 'data':falses_data}, num_lines = 10)

Mac Akasaka, rare earths trader, perennial candidate in Japanese elections Ichiro Fujisaki, former Japanese ambassador to the United States, currently member of the Board of Directors of Itochu Hiroyuki Nagahama, member of the House of Representatives and Environment Minister in 2012, worked for Itochu early in his professional career Uichiro Niwa, president from 1998 to 2004, later Japanese ambassador to China Ry*z* Sejima, chairman from 1978 to 1981, Kwantung Army staff officer during World War II Toshiyuki Takano, retired diplomat, currently executive advisor to Itochu Tsuneharu Takeda, former Imperial prince and later ambassador to Bulgaria, worked at Itochu from 1967 to 2005 and served as head of its subsidiaries in Australia and New Zealand.

In this episode, his character's last name is Peters (later Hale). With Becky going back to work, Jesse becomes ``Mr. Mom'', while also trying to pursue a record deal. Meanwhile, Danny deals with Vicki leaving, and Stephanie accidentally sta

## 4. When the pronoun is subject pronoun

### * And when only one word is subject

In [28]:
subject_data = train_data[train_data['Pronoun'].isin(['he', 'she', 'He', 'She'])]
len(subject_data)

801

In [29]:
print_sents(datas={'manual':True, 'data':subject_data}, num_lines = 10)

Carmen Frei Ruiz-Tagle (born June 22, 1938) is a Chile an politician and teacher. She served as Senator from 1990 to 2006. Frei was born in Santiago, the eldest child to Eduardo Frei Montalva and Mar*a Ruiz-Tagle Jim*nez. She studied at the Colegio Universitario Ingl*s; and later graduated as a teacher (specializing in infants) from the Universidad de Chile.

During the next two years she was in the chorus of the Vari*t*s and the Folies Berg*re. In 1905 Delysia was one of the French ``Gibson Girls'' in The Catch of the Season at Daly's Theatre, New York, with Edna May in the leading role. She moved to London, where she continued to appear on the stage until 1909, when she temporarily abandoned the theatrical profession.

She is best known for a series of well-regarded television roles including Virginia Braithwaite, daughter of a lottery winning family in the comedy drama At Home with the Braithwaites. Sparkhouse (Red Production Company / BBC 2002); and her appearance in Jane Hall (Red

In [30]:
subject_data.iloc[204]

ID                                                  development-509
Text              At his trial Jonah Anguka had based his defenc...
Pronoun                                                          he
Pronoun-offset                                                  583
A                                                             Cohen
A-offset                                                        479
A-coref                                                       False
B                                                            Anguka
B-offset                                                        510
B-coref                                                        True
URL                       http://en.wikipedia.org/wiki/Jonah_Anguka
Name: 508, dtype: object

In [31]:
print(subject_data.iloc[203]['Text'])
words, tagged_words, word_index = tokenized_word_sets(row)
print(tagged_words)

By the 1970s Bergman had already received two Academy Awards from five nominations, but went on to be nominated twice more, winning for a third time, this time in the category of Best Supporting Actress, for 1974's Murder on the Orient Express, based on the Agatha Christie novel of the same name, for which she also received her first and only BAFTA.
[('Watkins', 'NNS'), ('was', 'VBD'), ('a', 'DT'), ('close', 'JJ'), ('friend', 'NN'), ('of', 'IN'), ('Hess', 'NNP'), ("'", 'POS'), ('first', 'JJ'), ('wife', 'NN'), (',', ','), ('Katherine', 'NNP'), ('Hawley', 'NNP'), ('.', '.'), ('She', 'PRP'), ('was', 'VBD'), ('one', 'CD'), ('of', 'IN'), ('the', 'DT'), ('first', 'JJ'), ('to', 'TO'), ('send', 'VB'), ('sympathy', 'NN'), ('when', 'WRB'), ('Hawley', 'NNP'), ('was', 'VBD'), ('thrown', 'VBN'), ('from', 'IN'), ('a', 'DT'), ('horse', 'NN'), ('and', 'CC'), ('killed', 'VBN'), ('.', '.'), ('Watkins', 'NNS'), ('and', 'CC'), ('Hess', 'NNP'), ('had', 'VBD'), ('a', 'DT'), ('son', 'NN'), (',', ','), ('Adam

In [47]:
def word_subject(row):
    raw = row['Text']
    words, tagged_words, word_index = tokenized_word_sets(row)

    A_id = row['A-offset']
    B_id = row['B-offset']

    A_tokens = len(row['A'].split())
    B_tokens = len(row['B'].split())

    if tagged_words[word_index[A_id]+A_tokens][1].startswith('V'):
        if tagged_words[word_index[B_id]+B_tokens][1].startswith('V'):
            return (True, True)
        else:
            return (True, False)
    else:
        if tagged_words[word_index[B_id]+B_tokens][1].startswith('V'):
            return (False, True)
        else:
            return (False, False)

def only_subject(row):
    return sum(word_subject(row))%2 == 1

In [48]:
only_subject_data = subject_data.apply(only_subject, axis=1)
only_subject_data = subject_data[only_subject_data]
len(only_subject_data)

447

In [49]:
only_subject_data['A-pred'] = only_subject_data.apply(lambda row: word_subject(row)[0], axis=1)
only_subject_data['B-pred'] = only_subject_data.apply(lambda row: word_subject(row)[1], axis=1)

<ipython-input-49-146d5bf28ab4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_subject_data['A-pred'] = only_subject_data.apply(lambda row: word_subject(row)[0], axis=1)
<ipython-input-49-146d5bf28ab4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_subject_data['B-pred'] = only_subject_data.apply(lambda row: word_subject(row)[1], axis=1)


In [50]:
accuracy(only_subject_data)

0.8165548098434005

### * And when both words are subject

In [51]:
def both_subject(row):
    return word_subject(row) == (True, True)

In [52]:
both_subject_data = subject_data.apply(both_subject, axis=1)
both_subject_data = subject_data[both_subject_data]
len(both_subject_data)

53

In [54]:
print_sents(datas={'manual':True, 'data':both_subject_data}, num_lines = 30)

Alice went to live with Nick's sister Kathy, who desperatley tried to contact Elsa but was shocked to discover she'd moved to Australia without telling anyone. Relising she was all Alice had left and just how selfish and uncaring Elsa was, Kathy took custody of Alice.

Nicholas Rogers as Tarabas role also caused problems, as has declared during his audition that he was able to ride horses but as soon as the shooting began, he could not even mount a horse. It was then soon realized he was also not good at acting; Bava said that for the scene where Rogers has to say ``I love you'', they had to make three shots and then combine them later. He also injured himself on the set and had to be taken to hospital while still wearing his costume.

On January 17, 2014, Brigman was selected in the second round (17th overall pick) of the 2014 NWSL College Draft by Seattle Reign FC. In March, the Reign signed her to the team. Of her signing, head coach Laura Harvey said, ``Megan was someone that we wa

Assume when both words are subject, closer word is True, the other is False
--> accuracy = 22/30 = 73.3%

### * And when both words are not subject

In [55]:
def none_subject(row):
    return word_subject(row) == (False, False)

In [56]:
none_subject_data = subject_data.apply(none_subject, axis=1)
none_subject_data = subject_data[none_subject_data]
len(none_subject_data)

301

In [57]:
print_sents(datas={'manual':True, 'data':none_subject_data}, num_lines = 10)

As part of his breeding and training programs he was part owner (with Harold Syner of Wajima fame) of Alamare Farm in Lexington, Kentucky and joint owner of Meadow Wood Training Center in Ocala, Florida. DiMauro is responsible for mentoring and developing many of the horsemen involved in Thoroughbred racing today. His experience and insight into the sport has earned him much respect among many. He gave Richard Migliore a chance, and Richard ``The Mig,'' was a well-known jockey for many years until his retirement in June 2010, and is still involved in the sport with the New York Racing Association.

Contemporary analysis reveals that Perot could have won the election if the polls prior to the election had shown the candidate with a larger share, preventing the wasted vote mindset. Republicans, including Vice President Dan Quayle, blamed Perot for their defeat, claiming that he won votes that would have otherwise gone to Bush.

In 1915, while studying at the Chicago Art Institute, he won